## Eigenfaces - Face Recognition

#### Importing necessary libraries

In [41]:
import os
import numpy as np
from PIL import Image

#### Loading and transforming the images into vectors

In [42]:
def load_images(path):
    images = {}
    for file in filter(lambda file: not file.startswith('.'), os.listdir(path)):
        img = Image.open(os.path.join(path, file))
        idx = int(file.split('_')[0])
        if not idx in images:
            images[idx] = []
        images[idx].append(np.asarray(img))

    return images

def transform_to_vec(dataset):
    for person_face in dataset:
        dataset[person_face] = list(map(lambda img: img.reshape(-1,1), dataset[person_face]))
    return dataset

def compute_average(dataset):
    average_face = {}
    for person_face in dataset:
        person_faces = dataset[person_face]
        average_face[person_face] = np.sum(person_faces, axis=0) / len(person_faces)
    return average_face

def normalize_dataset(dataset, mean):
    for person_face in dataset:
        dataset[person_face] = list(map(lambda face: face - mean[person_face],dataset[person_face]))
    return dataset

def compute_covariance_matrix(dataset):
    cov_matrixes = {}
    for person_face in dataset:
        matrixes = list(map(lambda face: face * face.T, dataset[person_face]))
        for matrix in matrixes:
            if person_face not in cov_matrixes:
                cov_matrixes[person_face] = matrix
            else:
                cov_matrixes[person_face] += matrix
        cov_matrixes[person_face] *=  1 / len(dataset[person_face])
    return cov_matrixes

def compute_face_covariance_matrix(face):
    return face * face.T

def compute_face_eigen_matrix(face, num_eigs=14):
    cov_matrix = compute_face_covariance_matrix(face)
    values, vects = np.linalg.eig(cov_matrix)
    indices = []
    while len(indices) < num_eigs:
        index = np.argmax(values)
        np.delete(values,index)
        indices.append(index)
    # print(vects.shape)
    # print(vects[:,indices].shape)
    return  vects[:,indices]


def compute_eigen_matrixes(cov_matrixes, num_eigs=14):
    eigen_matrixes = {}
    for matrix in cov_matrixes:
        values, vects = np.linalg.eig(cov_matrixes[matrix])
        indices = [i for i,v in enumerate(values > 0.5) if v]
        indices = indices[:num_eigs]
        eigen_matrixes[matrix] =  vects[:,indices]
    return eigen_matrixes

        
def compute_projection(image, eigen_matrix):
    # print('projection')
    # print(image.shape, eigen_matrix.shape)
    return np.asarray(eigen_matrix) * image.reshape(-1,1)

##### Computing the eigen vectors

In [43]:
DATA_PATH = '../data'
TRAIN_DATA_PATH = os.path.join(DATA_PATH, 'train')
TEST_DATA_PATH = os.path.join(DATA_PATH, 'test')

train_data = load_images(TRAIN_DATA_PATH)
train_data = transform_to_vec(train_data)

mean_train_faces = compute_average(train_data)
train_data = normalize_dataset(train_data, mean_train_faces)
cov_matrices = compute_covariance_matrix(train_data)
eigen_matrixes = compute_eigen_matrixes(cov_matrices)

##### Computing the test dataset performance


In [44]:
test_data = load_images(TEST_DATA_PATH)
test_data = transform_to_vec(test_data)
test_data = normalize_dataset(test_data, mean_train_faces)
wrong = 0
correct = 0

for person_face in test_data:
    for face in test_data[person_face]:
        min_diff = None
        prediction = None
        for matrix in eigen_matrixes:
            projected_face = compute_projection(face, eigen_matrixes[matrix])
            for seen_face in train_data[matrix]:
                projection = compute_projection(seen_face, eigen_matrixes[matrix])
                diff = np.linalg.norm(projection - projected_face)
                if min_diff is None:
                    min_diff = diff
                    prediction = matrix
                elif min_diff > diff:
                    min_diff = diff
                    prediction = matrix
        if prediction == person_face:
            correct += 1
        else:
            wrong += 1
        print('Predicted: {} - Real: {}'.format(prediction, person_face))

print('Correct: {} - Wrong: {} - Accuracy: {:.2f}%'.format(correct, wrong, correct * 100 / (correct + wrong)))
            



Predicted: 1 - Real: 1
Predicted: 1 - Real: 1
Predicted: 2 - Real: 1
Predicted: 2 - Real: 1
Predicted: 2 - Real: 1
Predicted: 2 - Real: 1
Predicted: 1 - Real: 1
Predicted: 4 - Real: 1
Predicted: 1 - Real: 1
Predicted: 1 - Real: 1
Predicted: 1 - Real: 1
Predicted: 1 - Real: 1
Predicted: 1 - Real: 1
Predicted: 5 - Real: 1
Predicted: 4 - Real: 1
Predicted: 1 - Real: 1
Predicted: 1 - Real: 1
Predicted: 1 - Real: 1
Predicted: 4 - Real: 1
Predicted: 4 - Real: 1
Predicted: 4 - Real: 1
Predicted: 4 - Real: 1
Predicted: 4 - Real: 1
Predicted: 1 - Real: 1
Predicted: 1 - Real: 1
Predicted: 1 - Real: 1
Predicted: 1 - Real: 1
Predicted: 2 - Real: 1
Predicted: 4 - Real: 1
Predicted: 2 - Real: 1
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 1 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 4 - Real: 2
Predicted: 2 - Real: 2
Predicted: 4 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 2 - Real: 2
Predicted: 